In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [2]:
import re
aspect_set = set()
with open("../Multiwoz/WOZ_dev_ans.txt") as f:
  for line in f:
    line = line.strip()
    line_lst = line.split("|")
    for aspect in line_lst[1:]:
      aspect_name = re.search(r"-(\w+)=(\w+)",aspect).group(1)
      aspect_set.add(aspect_name)


In [3]:
aspect_set.remove('name')
aspect_set.remove('food')

In [4]:
aspect_set

{'area', 'internet', 'parking', 'pricerange', 'stars', 'type'}

In [7]:
aspect_path = "./data/aspect/"

In [12]:
from collections import defaultdict
aspect_dict = defaultdict(set)
for aspect in list(aspect_set):
  with open(aspect_path + aspect + "_train.tsv", "w") as fout:
    with open("../Multiwoz/WOZ_train_ans.txt") as f_ans:
      with open("../Multiwoz/WOZ_train_utt.txt") as f_utt:
        fout.write("content\tlabel\n")
        for line_ans, line_utt in zip(f_ans, f_utt):
          line_ans = line_ans.strip()
          line_utt = line_utt.strip()
          line_lst = line_ans.split("|")
          label = "None"
          for aspect_item in line_lst[1:]:
            match = re.search(r"-(\w+)=(\w+)",aspect_item)
            if match and match.group(1) == aspect:
              label = match.group(2)
              aspect_dict[aspect].add(label)
          fout.write(line_utt + "\t" + label + "\n")


In [11]:
for aspect in list(aspect_set):
  with open(aspect_path + aspect + "_dev.tsv", "w") as fout:
    with open("../Multiwoz/WOZ_dev_ans.txt") as f_ans:
      with open("../Multiwoz/WOZ_dev_utt.txt") as f_utt:
        fout.write("content\tlabel\n")
        for line_ans, line_utt in zip(f_ans, f_utt):
          line_ans = line_ans.strip()
          line_utt = line_utt.strip()
          line_lst = line_ans.split("|")
          label = "None"
          for aspect_item in line_lst[1:]:
            match = re.search(r"-(\w+)=(\w+)",aspect_item)
            if match and match.group(1) == aspect:
              label = match.group(2)
          fout.write(line_utt + "\t" + label + "\n")

In [10]:
aspect_dict

defaultdict(set,
            {'internet': {'dontcare', 'no', 'yes'},
             'parking': {'dontcare', 'no', 'yes'},
             'pricerange': {'cheap', 'dontcare', 'expensive', 'moderate'},
             'area': {'centre', 'dontcare', 'east', 'north', 'south', 'west'},
             'type': {'dontcare', 'guesthouse', 'hotel'},
             'stars': {'0', '1', '2', '3', '4', '5', 'dontcare'}})